# Transformação de Dados

Nos notebooks anteriores, aprendemos a usar marcas e codificações visuais para representar registros individuais de dados. Aqui, exploraremos métodos para *transformar* dados, incluindo o uso de agregações para resumir múltiplos registros. A transformação de dados é uma parte essencial da visualização: escolher quais variáveis exibir e seu nível de detalhamento é tão importante quanto escolher codificações visuais apropriadas. Afinal, não importa quão bem escolhidas sejam suas codificações visuais se você estiver mostrando as informações erradas!

À medida que você avança neste módulo, recomendamos que abra a [documentação de Transformações de Dados do Altair](https://altair-viz.github.io/user_guide/transform/index.html) em outra aba. Ela será um recurso útil caso você queira mais detalhes ou explorar outras transformações disponíveis.

_Este notebook faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._


In [ ]:
import pandas as pd
import altair as alt

: 

## A tabela de dados sobre filmes

Trabalharemos com uma tabela de dados (_dataset_) sobre filmes, extraída da coleção [vega-datasets](https://vega.github.io/vega-datasets/). Os dados incluem variáveis como o nome do filme, diretor, gênero, data de lançamento, classificações e receitas brutas. No entanto, _tenha cuidado ao trabalhar com esses dados_: os filmes são de anos com amostras desiguais usando dados combinados de múltiplas fontes. Se você analisar com mais detalhe, encontrará problemas como valores ausentes e até alguns erros sutis! Mesmo assim, os dados devem ser interessantes para explorar...

Vamos obter a URL do arquivo JSON de dados a partir do pacote vega_datasets e, em seguida, carregar os dados em um DataFrame do Pandas para que possamos inspecionar seu conteúdo.


In [ ]:
movies_url = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
movies = pd.read_json(movies_url)

Quantas linhas (registros) e colunas (campos) existem no _dataset_ de filmes?

In [ ]:
movies.shape

Agora, vamos dar uma olhada nas primeiras 5 linhas da tabela para entender melhor os campos e os tipos de dados...

In [ ]:
movies.head(5)

## Histogramas

Vamos começar nosso tour pelas tranformações _dividindo_ os dados em gupos discretos e _contando_ os registros para resumir esses grupos. Os gráficos que resultam disso são conhecidos como [_histogramas_](https://pt.wikipedia.org/wiki/Histograma).

Primeiro, vamos observar os dados não agregados: um gráfico de dispersão mostrando avaliações de filmes do site Rotten Tomatoes versus as avaliações de usuários da plataforma IMDB. Vamos providenciar o dados ao Altair passando a URL dos dados do filme para o método `Chart`. (Nós também poderíamos passar o _data frame_ do Pandas diretamente para conseguirmos o mesmo resultado.) Podemos então diferenciar as avaliações do Rotten Tomatoes e IMDB utilizando os canais `x` e `y`:

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q'),
    alt.Y('IMDB_Rating:Q')
)

Para resumir esses dados, podemos separar um campo de dados para agrupar valores numéricos em grupos discretos. Aqui, nós separamos no eixo x utilizando o parâmetro `bin=True` no canal de codificação `x`. O resultado é um conjunto de dez partições com mesmo tamanho de intervalo, cada uma correspondendo a uma gama de dez pontos de avaliação diferentes.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Y('IMDB_Rating:Q')
)

Configurar `bin=True` utiliza a configuração padrão de particionamento, porém, conseguimos ter mais controle sobre esse parâmetro se desejarmos. Vamos ajustar a contagem máxima de partições (`maxbins`) para 20, a qual tem o efeito de dobrar o número de partições. Agora cada parte corresponde a uma gama de cinco pontos de avaliação.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('IMDB_Rating:Q')
)

Com os dados repartidos, vamos agora resumir a distribuição das avalições do Rotten Tomatoes. Vamos deixar de lado as avaliações do IMDB por enquanto, e no lugar disso, vamos usar o canal de codificação `y` para exibir uma contagem (`count`) agregada dos registros, de modo que a posição vertical de cada ponto indica o número de filmes por avaliação (considerando as partições) do Rotten Tomatoes.

Como o agregador `count` conta o número total de registros em cada repartição indepedentemente do valor do campo, não precisamos incluir um nome de campo na codificação `y`.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count()')
)

Para chegarmos em um histograma padrão, vamos mudar o tipo de marca de círculo (`circle`) para barra (`bar`):

In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count()')
)

_Podemos agora examinar a distribuição de avaliações mais claramente: podemos ver menos filmes na faixa negativa e um pouco mais de filmes com notas maiores, mas de maneira geral obsevamos uma distribuição uniforme. Avaliações do Rotten Tomatoes são determinadas utilizando "joinha" e "deslike" dos críticos de filme e calculando a porcentagem de reações positivas. Essa abordagem aparenta realizar um bom trabalho utilizando toda faixa dos valores das avaliações._

Similarmente, podemos criar um histograma para as avaliações do IMDB trocando apenas o campo no canal de codificação `x`: 

In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('IMDB_Rating:Q', bin=alt.BinParams(maxbins=20)),
    alt.Y('count()')
)

_Em contraste com a distribuição uniforme que vimos antes, as classificações do IMDB exibem uma distribuição em forma de sino (embora esteja [negativamente distorcida](https://en.wikipedia.org/wiki/Skewness)). As classificações do IMDB são formadas pela média das pontuações (que variam de 1 a 10) fornecidas pelos usuários do site. Podemos ver que esta forma de medição leva a um formato diferente das classificações do Rotten Tomatoes. Podemos notar também que a moda da distribuição está entre 6.5 e 7: as pessoas geralmente gostam de ver filmes, o que explica potencialmente o viés positivo!_

Agora devemos voltar ao nosso gráfico de dispersão das classificações do Rotten Tomatoes e do IMDB. Aqui está o que acontece se descartarmos _ambos_ os eixos do nosso gráfico original.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin = alt.BinParams(maxbins = 20)),
    alt.Y('IMDB_Rating:Q', bin = alt.BinParams(maxbins = 20))
)

Os detalhes são perdidos devido ao _excesso de plotagens_, onde muitos pontos estão desenhados diretamente uns sobre os outros.

Para formar um histograma bidimensional, podemos adicionar um agregador `count` como antes. Como os canais de codificação `x` e `y` já estão em uso, devemos usar um canal de codificação diferente para transmitir as contagens. Aqui está o resultado do uso de área circular, adicionando um canal de codificação de tamanho (`size`).

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin = alt.BinParams(maxbins = 20)),
    alt.Y('IMDB_Rating:Q', bin = alt.BinParams(maxbins = 20)),
    alt.Size('count()')
)

De forma alternativa, podemos codificar contagens usando o canal cor (`color`) e alterar o tipo de marca para barra (`bar`). O resultado é um histograma bidimensional na forma de um _[heatmap](https://en.wikipedia.org/wiki/Heat_map)_.

In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('Rotten_Tomatoes_Rating:Q', bin = alt.BinParams(maxbins = 20)),
    alt.Y('IMDB_Rating:Q', bin = alt.BinParams(maxbins = 20)),
    alt.Color('count()')
)

Compare os histogramas 2D baseados em tamanho e cor acima. Qual codificação você acha que deveria ser preferida? Por que? Em qual gráfico você pode comparar com mais precisão a magnitude dos valores individuais? Em qual gráfico você consegue ver com mais precisão a densidade geral das avaliações?

## Agregação

As contagens são apenas um tipo de resumo. Podemos também calcular resumos usando medidas como a média (`average`), mediana (`median`), mínimo (`min`), ou máximo (`max`). A documentação do Altair inclui o [conjunto completo de funções de resumo disponíveis](https://altair-viz.github.io/user_guide/transform/aggregate.html#user-guide-aggregate-transform).

Vamos dar uma olhada em alguns exemplos!

### Médias e ordenação

_Diferentes gêneros de filmes recebem de forma consistente diferentes classificações dos críticos?_ Como um primeiro passo para responder a esta pergunta, podemos examinar a [média (também conhecida como *média aritmética*)](https://en.wikipedia.org/wiki/Arithmetic_mean) de classificação para cada gênero de filme.

Vamos visualizar o gênero ao longo do eixo ``y`` e traçar a classificação média do Rotten Tomatoes ao longo do eixo ``x``.


In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N')
)

_Parece haver alguma variação interessante, mas olhar para os dados como uma lista alfabética não é muito útil para classificar as respostas da crítica aos gêneros._

Para uma imagem mais organizada, vamos ordenar os gêneros por ordem decrescente de classificação média. Para fazermos isso, vamos adicionar um parâmetro ordem
(`sort`) ao canal de codificação `y`, indicando que queremos ordenar pela média (`op`, a operação de agregação) da classificação do *Rotten Tomatoes* (o 
`field`) por ordem (`order`) decrescente.

In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N', sort=alt.EncodingSortField(
        op='average', field='Rotten_Tomatoes_Rating', order='descending')
    )
)

_A ordenação sugere que os críticos apreciam muito os documentários, os musicais, os faroestes e os dramas, mas desprezam as comédias românticas e os filmes de terror... e quem não gosta de filmes com gênero desconhecido (`null`)?_

<!--### Medians and the Inter-Quartile Range

While averages are a common way to summarize data, they can sometimes mislead. For example, very large or very small values ([*outliers*](https://en.wikipedia.org/wiki/Outlier)) might skew the average. To be safe, we can compare the genres according to the [*median*](https://en.wikipedia.org/wiki/Median) ratings as well.

The median is a point that splits the data evenly, such that half of the values are less than the median and the other half are greater. The median is less sensitive to outliers and so is referred to as a [*robust* statistic](https://en.wikipedia.org/wiki/Robust_statistics). For example, arbitrarily increasing the largest rating value will not cause the median to change.

Let's update our plot to use a `median` aggregate and sort by those values:-->

### Medianas e intervalo interquartil

Embora as médias sejam uma forma comum de resumir dados, às vezes podem ser enviesadas. Por exemplo, valores muito grandes ou muito pequenos ([*outliers*](https://en.wikipedia.org/wiki/Outlier)) podem distorcer a média. Por segurança, também podemos comparar os dados de acordo com a [*mediana*](https://en.wikipedia.org/wiki/Median) das classificações.

A mediana é um ponto que divide os dados uniformemente, de forma que metade dos valores seja menor que a mediana e a outra metade seja maior. A mediana é menos sensível a valores discrepantes e, portanto, é chamada de [estatística *robusta*](https://en.wikipedia.org/wiki/Robust_statistics). Por exemplo, aumentar arbitrariamente o maior valor de classificação não fará com que a mediana mude.

Vamos atualizar nosso gráfico para usar um agregador de mediana (`median`) e classificar por esses valores:

In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('median(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N', sort=alt.EncodingSortField(
        op='median', field='Rotten_Tomatoes_Rating', order='descending')
    )
)

<!--_We can see that some of the genres with similar averages have swapped places (films of unknown genre, or `null`, are now rated highest!), but the overall groups have stayed stable. Horror films continue to get little love from professional film critics._

It's a good idea to stay skeptical when viewing aggregate statistics. So far we've only looked at *point estimates*. We have not examined how ratings vary within a genre.

Let's visualize the variation among the ratings to add some nuance to our rankings. Here we will encode the [*inter-quartile range* (IQR)](https://en.wikipedia.org/wiki/Interquartile_range) for each genre. The IQR is the range in which the middle half of data values reside. A [*quartile*](https://en.wikipedia.org/wiki/Quartile) contains 25% of the data values. The inter-quartile range consists of the two middle quartiles, and so contains the middle 50%. 

To visualize ranges, we can use the `x` and `x2` encoding channels to indicate the starting and ending points. We use the aggregate functions `q1` (the lower quartile boundary) and `q3` (the upper quartile boundary) to provide the inter-quartile range. (In case you are wondering, *q2* would be the median.)-->

_Podemos ver que alguns dos gêneros com médias semelhantes trocaram de lugar (filmes de gênero desconhecido, ou ‘null’, agora têm as classificações mais altas!), mas os grupos gerais permaneceram estáveis. Os filmes de terror continuam a receber pouco entusiasmo dos críticos de cinema profissionais._

É uma boa ideia permanecer cético ao visualizar estatísticas agregadas. Até agora, analisamos apenas *estimativas locais*. Não examinamos como as classificações variam dentro de um gênero.

Vamos visualizar a variação entre as classificações para adicionar algumas nuances às nossas classificações. Aqui codificaremos o [*intervalo interquartil* (IQR)](https://en.wikipedia.org/wiki/Interquartile_range) para cada gênero. O IQR é o intervalo em que reside a metade intermediária dos valores dos dados. Um [*quartil*](https://en.wikipedia.org/wiki/Quartile) contém 25% dos valores dos dados. O intervalo interquartil consiste nos dois quartis intermediários e, portanto, contém os 50% intermediários. 

Para visualizar intervalos, podemos usar os canais de codificação `x` e `x2` para indicar os pontos inicial e final. Usamos as funções agregadoras `q1` (o limite do quartil inferior) e `q3` (o limite do quartil superior) para fornecer o intervalo interquartil. (Caso você esteja se perguntando, *q2* seria a mediana.)

In [ ]:
alt.Chart(movies_url).mark_bar().encode(
    alt.X('q1(Rotten_Tomatoes_Rating):Q'),
    alt.X2('q3(Rotten_Tomatoes_Rating):Q'),
    alt.Y('Major_Genre:N', sort=alt.EncodingSortField(
        op='median', field='Rotten_Tomatoes_Rating', order='descending')
    )
)

### Unidades de Tempo

_Agora vamos fazer uma pergunta completamente diferente: as bilheterias variam de acordo com a estação do ano?_

Para conseguir a resposta inicial, vamos traçar a média bruta de lucro nos Estados Unidos por mês.

Para fazer esse gráfico, use a unidade de tempo (`timeunit`) para mapear as datas de lançamento por mês (`month`) do ano. O resultado é similar a discretização, mas usando intervalos de tempo mais significativos. Outras unidades de tempo incluem ano (`year`), trimestre (`quarter`), dia do mês (`date`), dia da semana (`day`) e horas (`hours`). Há também outras unidades, chamadas compostas, como ano-mês (`yearmonth`) ou horas-minutos (`hoursminutes`). Veja a documenteção do Altair para [lista completa de unidades de tempo](https://altair-viz.github.io/user_guide/transform/timeunit.html#user-guide-timeunit-transform).



In [ ]:
alt.Chart(movies_url).mark_area().encode(
    alt.X('month(Release_Date):T'),
    alt.Y('median(US_Gross):Q')
)

_Observando o gráfico resultante, as vendas médias de filmes nos EUA parecem aumentar em torno da temporada de sucessos de bilheteria do verão e do período de férias de fim de ano. Claro, pessoas ao redor do mundo (não apenas os EUA) vão ao cinema. Um padrão semelhante surge para a receita bruta ao redor do mundo?_

In [ ]:
alt.Chart(movies_url).mark_area().encode(
    alt.X('month(Release_Date):T'),
    alt.Y('median(Worldwide_Gross):Q')
)

_Sim!_

## Transformação avançada de dados

Os exemplos acima usam transformações (*bin*, *timeunit*, *aggregate*, *sort*) que são definidas em relação a um canal de codificação. Porém, em algumas situações você talvez queira aplicar uma cadeia de múltiplas transformações antes da visualização, ou usar transformações que não se integram em definições de codificação. Para tais casos, Altair e Vega-Lite suportam transformações de dados definidas separadamente das codificações. Essas transformações são aplicadas nos dados *antes* de qualquer codificação ser considerada.

Nós também *poderíamos* fazer transformações usando Pandas diretamente, e então visualizar o resultado. Porém, usar as transformações embutidas permite que nossas visualizações sejam publicadas mais facilmente em outros contextos; por exemplo, exportar o Vega-Lite JSON para usar em uma interface da web independente. Vamos dar uma olhada nas transformações embutidas suportadas pelo Altair como calcular (`calculate`), filtrar (`filter`), agregar (`aggregate`), e janela (`window`).


### *Calcular (`calculate`) *

_Pense novamente na comparação entre a receita nos E.U.A e a receita mundial. A receita mundial não inclui os EUA? (De fato, inclui) Como podemos ter uma melhor noção das tendências fora dos EUA?_

Com a transformação `calculate` nós podemos derivar novos campos. Aqui queremos subtrair a receita dos E.U.A (`US_Gross`) da receita mundial (`Worldwide_gross`). A transformação `calculate` pega uma *string* de expressão Vega ([*Vega expression*](https://vega.github.io/vega/docs/expressions/)) para realizar uma operação sobre um único registro. *Vega expressions* utiliza sintaxe *JavaScript*. O prefixo `datum.` acessa os valores de um campo de regisro.

In [ ]:
alt.Chart(movies).mark_area().transform_calculate(
    NonUS_Gross='datum.Worldwide_Gross - datum.US_Gross'
).encode( 
    alt.X('month(Release_Date):T'),
    alt.Y('median(NonUS_Gross):Q')
)

_Podemos observar que as tendências sazonais se mantêm fora dos EUA, mas com um declínio significativo nos meses fora dos picos._

### Filtrar (`filter`)

A transformação *filter* cria uma nova tabela com um subconjunto dos dados originais, removendo linhas que não atendem a um requisito, conhecido como predicado ([*predicate*](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)). Semelhante à transformação *calculate*, os *filter predicates* são aplicados utilizando a linguagem [Vega expression](https://vega.github.io/vega/docs/expressions/).

Abaixo, adicionamos um filtro para limitar nosso gráfico de dispersão inicial de classificações do IMDB vs. Rotten Tomatoes para apenas filmes do gênero principal "Comédia Romântica" (*Romantic Comedy*).

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q'),
    alt.Y('IMDB_Rating:Q')
).transform_filter('datum.Major_Genre == "Romantic Comedy"')

_Como o gráfico muda se observarmos os outros gêneros? Edite a expressão do filtro para descobrir._

Agora, vamos filtrar para ver apenas filmes lançados antes de 1970.

In [ ]:
alt.Chart(movies_url).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating:Q'),
    alt.Y('IMDB_Rating:Q')
).transform_filter('year(datum.Release_Date) < 1970')

_Eles parecem ter pontuações anormalmente altas! Filmes antigos são melhores que os demais ou há um viés de seleção ([selection bias](https://en.wikipedia.org/wiki/Selection%5Fbias)) a filmes mais antigos com melhor classificação neste conjunto de dados?_

### Agregação (`aggregate`)

Já vimos transformações com agregação (`aggregate`), como `count` e `average`, no contexto de canais de codificação. Também podemos especificar agregações separadamente, como uma etapa de pré-processamento para outras transformações (como nos exemplos da transformação `window` abaixo). A saída de uma transformação `aggregate` é uma nova tabela de dados com registros que contêm tanto os campos de `groupby` quanto as medidas agregadas calculadas.

Vamos recriar nosso gráfico de avaliações médias por gênero, mas desta vez utilizando uma transformação `aggregate` separada. A tabela de saída da transformação `aggregate` contém 13 linhas, uma para cada gênero.

Para ordenar o eixo `y`, devemos incluir uma operação de agregação obrigatória nas instruções de ordenação. Aqui utilizamos o operador `max`, que funciona bem porque há apenas um registro de saída por gênero. Poderíamos usar o operador `min` da mesma forma e obteríamos o mesmo gráfico.

In [ ]:
alt.Chart(movies_url).mark_bar().transform_aggregate(
    groupby=['Major_Genre'],
    Average_Rating='average(Rotten_Tomatoes_Rating)'
).encode(
    alt.X('Average_Rating:Q'),
    alt.Y('Major_Genre:N', sort=alt.EncodingSortField(
        op='max', field='Average_Rating', order='descending'
      )
    )
)

### Janela (`window`)
Transformações `window` realizam cálculos sobre grupos ordenados de dados. Estas transformações são ferramentas poderosas, capazes de realizar operações como ordenação, análise de avanço/atraso dos dados, soma cumulativa, somas correntes e média. Valores calculados a partir de uma transformação `window` são salvos na tabela de dados original como um novo campo de dados. As operações *window* incluem as operações de agregação vistas anteriormente, bem como operações especializadas, como `rank`, `row_number`, `lead`, e `lag`. A documentação do Vega-Lite lista [todas as operações *window* válidas](https://vega.github.io/vega-lite/docs/window.html#ops).

Uma utilidade para uma operação `window` é calcular os k primeiros elementos de uma lista. Vamos fazer um gráfico dos 20 maiores diretores em termos de lucro ao redor do mundo.

Primeiro, utilizamos o operador `filter` para remover registros em que não sabemos o diretor. Caso contrário o diretor `null` iria dominar a lista! Em seguida, aplicamos o `aggregate` para somar o lucro de cada filme, agrupado por diretor. Neste momento, poderiamos fazer um gráfico de barras ordenado, mas acabaríamos com centenas e centenas de diretores. Como podemos limitar a visualização a apenas os 20 primeiros?

A transformação `window` nos permite determinar os primeiros diretores calculando sua ordem relativa. Dentro da nossa definição da transformação `window`, podemos ordenar por lucro e usar a operação `rank` pra calcular as posições relativas àquela ordenação. Depois, podemos adicionar uma transformação `filter` para limitar os dados apenas aos registros cuja a posição é menor ou igual a 20.

In [ ]:
alt.Chart(movies_url).mark_bar().transform_filter(
    'datum.Director != null'
).transform_aggregate(
    Gross='sum(Worldwide_Gross)',
    groupby=['Director']
).transform_window(
    Rank='rank()',
    sort=[alt.SortField('Gross', order='descending')]
).transform_filter(
    'datum.Rank < 20'
).encode(
    alt.X('Gross:Q'),
    alt.Y('Director:N', sort=alt.EncodingSortField(
        op='max', field='Gross', order='descending'
    ))
)

_Vemos que Steven Spielberg foi muito bem sucedido em sua carreira! Entretanto, mostrar as somas pode favorecer diretores que tiveram carreiras mais longas, e portanto tiveram lucros maiores com a produção de mais filmes. O que aconteceria se nós trocassemos a operação de agregação escolhida? Quem é um diretor mais bem sucedido em termos da **media** ou **mediana** dos lucros por filme? Modifique a transformação agregada acima!_

Anteriormente neste notebook, trabalhamos com histogramas, que aproximam a [*função densidade de probabilidade*](https://en.wikipedia.org/wiki/Probability_density_function) de um conjunto de valores. Uma abordagem complementar é observar a [*distribuição cumulativa*](https://en.wikipedia.org/wiki/Cumulative_distribution_function). Por exemplo, considere um histograma no qual cada _bin_ inclui não só o seu valor mas também a soma dos valores de todos os _bins_ anteriores &mdash; o resultado é uma _soma corrente_, com o último _bin_ contando o número total de registros. Um gráfico cumulativo nos mostra diretamente, dado um valor de referência, quantos dos valores nos dados são menores ou iguais à referência.

Para um exemplo concreto, vamos observar a distribuição cumulativa de filmes por tempo de duração (em minutos). Apenas um subconjunto dos registros inclui a informação sobre o tempo de duração, portanto devemos primeiro aplicar `filter` aqueles que a possuem. Em seguida, aplicamos o `aggregate` para contar o número de filmes por duração (utilizando implicitamente _bins_ de tamanho 1 minuto). Depois, utilizamos a transformação `window` para computar a soma currente do total sobre os _bins_, em ordem crescente de tempo de duração.

In [ ]:
alt.Chart(movies_url).mark_line(interpolate='step-before').transform_filter(
    'datum.Running_Time_min != null'
).transform_aggregate(
    groupby=['Running_Time_min'],
    Count='count()',
).transform_window(
    Cumulative_Sum='sum(Count)',
    sort=[alt.SortField('Running_Time_min', order='ascending')]
).encode(
    alt.X('Running_Time_min:Q', axis=alt.Axis(title='Duration (min)')),
    alt.Y('Cumulative_Sum:Q', axis=alt.Axis(title='Cumulative Count of Films'))
)

_Vamos análisar a distribuição cumulativa da duração dos filmes. Podemos ver que os filmes com menos de 110 minutos compõem cerca de metade dos filmes para os quais temos a informação de duração. Podemos ver um crescimento constante entre os filmes de 90 minutos até 2 horas de duração, a partir do qual a distribuição começa a estabilizar. Apesar de raros, o dataset contém diversos filmes com mais de 3 horas de duração!_



## Resumo 

 Nesta seção, apresentamos apenas uma prévia do poder da transformação de dados! Para mais detalhes, incluindo todas as transformações disponíveis e seus parâmetros, veja a [ documentação de transformação de dados do Altair](https://altair-viz.github.io/user_guide/transform/index.html).

Às vezes, você precisará executar uma transformação de dados significativa para preparar seus dados _antes mesmo_ de usar ferramentas de visualização. Para se envolver com preparação de dados ([_data wrangling_](https://en.wikipedia.org/wiki/Data_wrangling)), através do próprio Python, você pode usar a [biblioteca Pandas](https://pandas.pydata.org/).
                                                                                                                                           

Primeiramente, devemos carregar os pacotes e os dados a serem trabalhados. Neste caso, o conjunto de dados utilizados foi o Gapminder, pois contém informações a respeito do desenvolvimento social de vários países: